# Notebook 4 – Next word steering

***

In this notebook we will showcase the effect of steering on the next word predicted using causal language models (generative LLMs trained to predict the next tokens given a sequence of previous tokens).

The notebook includes these :

1. [Displaying the next token from a prompt](#display-top-next-token---based-on-a-prompt)
2. [Steering the prompt using a steering vector, displaying top tokens now](#display-top-next-tokens---after-steering)
3. [Generating longer text based on a prompt and steering](#generate-longer-strings-of-text)

In [1]:
# Append the path to the Functions directory

import sys
sys.path.append('../Functions')

## Functions used in this Notebook:

### From "Next_word_steering.py":
- [initialize_model_and_tokenizer](#importing-python-functions-and-data) - Set the model and tokenizer
- [display_next_tokens](#display-top-next-token---based-on-a-prompt) - Print next token predictions
- [get_embedding_gpt](#create-steering-vector---using-own-sentences) - Embed a list of strings (create a steering vector)
- [get_steering_vector_gpt](#create-steering-vector---using-a-feature-file) - Create steering vector based on a feature file
- [display_steered_next_tokens](#display-top-next-tokens---after-steering) - Print next token predictions after steering
- [generate_steered_text](#generate-longer-strings-of-text) - Generate longer strings of text based on predictions and steering


***

## Import python functions and set model

In [2]:
from Next_word_steering import display_next_tokens, display_steered_next_tokens, get_embedding_gpt, get_steering_vector_gpt, initialize_model_and_tokenizer, generate_steered_text

In [3]:
model_name = "openai-community/gpt2"

model, tokenizer = initialize_model_and_tokenizer(model_name)

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


***

## Display top next token - based on a prompt

Input any sentence as the prompt

In [4]:
prompt = "I am a woman, my doctor is a"

display_next_tokens(model, tokenizer, prompt)


Prompt: 'I am a woman, my doctor is a'

Top 10 next-token predictions:

' man': 0.4267
' woman': 0.3539
' doctor': 0.0781
' physician': 0.0069
' feminist': 0.0069
' male': 0.0062
' lady': 0.0052
' Muslim': 0.0031
' gentleman': 0.0030
' girl': 0.0024


## Create steering vector - using own sentences

`get_embedding_gpt` creates an embedding of a list of strings, in the layer wanted.

- `normalize (True/False)` normalizes the embedding

In [5]:
layer_to_steer = 10
steering_coefficient = 1

# Example of strings: Women
steering_sentences = [
    "She braided her daughter’s hair with one hand while sending an email with the other — and no one questioned it.",
    "The midwife stood calm as storms, her voice steadier than the monitors beeping beside her.",
    "Wearing heels or combat boots, she walks like the world owes her space — and it does.",
    "She bleeds monthly and still runs marathons, meetings, and entire households.",
    "The senator adjusted her blazer and silenced the room before saying a single word.",
    "She is the matriarch, the memory-keeper, the one everyone calls when things fall apart.",
    "Her lipstick is warpaint, and her silence is strategy.",
    "From nursery rhymes to protest chants, her voice has always carried more than melody.",
    "She stitched every family story into the quilt that now warms three generations.",
    "She grew life inside her, lost sleep for years, and still built a business from scratch.",
    "The grandmother who crossed borders with babies strapped to her chest — that’s who she is.",
    "She is the girl told to smile, the teen told to shrink, the woman who refused.",
    "Behind every medal, there's a ponytail soaked in sweat and defiance.",
    "She signs her name where others once wrote hers for her.",
    "You can find her in every history book margin — not because she wasn’t there, but because someone tried to erase her."]

steering_vector = get_embedding_gpt(model, tokenizer, steering_sentences, layer_to_steer, normalize=True)

## Create steering vector - using a feature file

`get_steering_vector_gpt` creates a steering vector using the feature wanted and the layer wanted. This function uses the function `get_embedding_gpt` to create the embedding vector of the text.

This function uses the function `import_feature_texts(f"Features/{feature}")`, and reqires the user to have a folder called "Features" with a collection of feature texts inside files called "feature.txt" and optionally "opposite.txt".

- `normalize` normalizes the steering vector (but the get_embedding function is never normalized at the same time)

In [6]:
layer_to_steer = 8
steering_coefficient = 2
feature = "Love"

steering_vector = get_steering_vector_gpt(model, tokenizer, feature, layer_to_steer, normalize=True)

Opposite file not found: Features/Love/opposite.txt


## Display top next tokens - after steering

`display_steered_next_tokens` steers the prompt using the steering vector, and displays the top "k" predictions for the next token.

In [7]:
display_steered_next_tokens(model, tokenizer, prompt, layer_to_steer, steering_vector, steering_coefficient, k=10)


Prompt: 'I am a woman, my doctor is a'
Steering coefficient: 2

Top 10 next-token predictions:

' man': 0.4220
' woman': 0.3566
' doctor': 0.0764
' feminist': 0.0069
' physician': 0.0068
' male': 0.0064
' lady': 0.0052
' Muslim': 0.0031
' gentleman': 0.0031
' girl': 0.0024


***

## Generate longer strings of text

In [8]:
prompt = "In the year 3000, humanity"
steering_vector = get_steering_vector_gpt(model, tokenizer, "War", 8, normalize=True)

`generate_steered_text` steers the prompt using the steering vector and then generates a longer sentence or text using the predictions for the next tokens

- `max_tokens (int)` is the maximum amount of tokens that can be generated

- `stop_token="."` can make the generation of tokens stop at a "." thus creating one sentence, if `stop_token=None`, it stops only when "max_tokens" is reached

- `temperature (float)` controls randomness: lower = more deterministic, higher = more creative
    - **1.0** = No change (baseline)
    - **<1.0** = Less random, sharpens distribution, picking more probable words
    - **>1.0** = More creative and diverse output, picks less probable words
    - **0** = Always picks most probable token

In [16]:
generate_steered_text(model, tokenizer, prompt, layer_to_steer, steering_vector, steering_coefficient, stop_token=".", max_tokens=20, temperature=1.2)

'In the year 3000, humanity got tired and died, like one running across a desert of paved, irradiated dirt and wiped us'